<a href="https://colab.research.google.com/github/tranly-dev/ustomer-churn-prediction./blob/main/ustomer-churn-prediction./notebooks/customer_bankingchurn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏦 Customer Churn Analysis
Phân tích dữ liệu khách hàng rời bỏ ngân hàng nhằm:
- Tìm hiểu đặc điểm nhóm khách hàng **churn** (rời đi) so với **non-churn**.
- Xây dựng nền tảng cho mô hình dự đoán churn.

## 1️⃣ Import libraries
Các thư viện chính:
- `pandas`, `numpy`: xử lý dữ liệu.
- `matplotlib`, `seaborn`: trực quan hóa dữ liệu.


In [ ]:
# 1. Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Hiển thị trực quan đẹp hơn
plt.style.use('seaborn-v0_8')
sns.set_palette("Set2")

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)


## 2️⃣ Load dataset
Dữ liệu nằm trong thư mục `../data/raw/Churn_Modelling.csv`.  
Mục tiêu: kiểm tra kích thước và xem qua vài dòng đầu.


In [ ]:
# 2. Load dataset
data = pd.read_csv('../data/raw/Churn_Modelling.csv')

print("Kích thước dữ liệu:", data.shape)
data.head()


## 3️⃣ Kiểm tra dữ liệu tổng quan
- Kiểm tra thông tin các cột (`info()`).
- Thống kê mô tả (`describe()`).
- Kiểm tra giá trị bị thiếu (`isnull()`).


In [ ]:
# 3. Kiểm tra dữ liệu tổng quan
data.info()
print("\nThống kê mô tả:")
display(data.describe())

# Kiểm tra giá trị null
print("\nSố lượng giá trị bị thiếu mỗi cột:")
print(data.isnull().sum())


## 4️⃣ Phân tích sơ bộ

Trong phần này, chúng ta thực hiện **phân tích khám phá dữ liệu (EDA)** để hiểu rõ hơn về đặc điểm khách hàng và hành vi rời bỏ (churn).

**Mục tiêu:**
- Tính tỷ lệ khách hàng churn tổng thể.
- Trực quan hóa churn theo **giới tính**, **quốc gia**, và **độ tuổi**.
- Quan sát phân phối các biến chính trong dataset.


In [ ]:
# 4. Phân tích sơ bộ
# Tỉ lệ churn
churn_rate = data['Exited'].value_counts(normalize=True)
print("Tỉ lệ churn:")
print(churn_rate)

# Biểu đồ phân bố churn
sns.countplot(x='Exited', data=data)
plt.title('Phân bố khách hàng churn vs giữ lại')
plt.show()

# Churn theo giới tính
sns.countplot(x='Gender', hue='Exited', data=data)
plt.title('Tỉ lệ churn theo giới tính')
plt.show()

# Churn theo quốc gia
sns.countplot(x='Geography', hue='Exited', data=data)
plt.title('Tỉ lệ churn theo quốc gia')
plt.show()

# Phân phối tuổi
sns.histplot(data['Age'], kde=True)
plt.title('Phân phối tuổi khách hàng')
plt.show()


## 5️⃣ Phân tích tương quan giữa các biến số

Mục tiêu:
- Xem xét mối tương quan giữa các biến định lượng (như `CreditScore`, `Age`, `Balance`, `EstimatedSalary`, …).
- Tìm hiểu những đặc điểm có thể ảnh hưởng mạnh đến biến mục tiêu `Exited`.

Chúng ta sẽ dùng **ma trận tương quan (correlation matrix)** và **biểu đồ nhiệt (heatmap)** để trực quan hóa.


In [ ]:
# 5. Phân tích tương quan giữa các biến số
corr = data.corr(numeric_only=True)
plt.figure(figsize=(10, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Ma trận tương quan giữa các biến số")
plt.show()


## 6️⃣ Nhận xét sơ bộ sau EDA

Dựa trên các biểu đồ và thống kê ở trên, ta rút ra một số nhận xét nhanh về dữ liệu.


In [ ]:
# 6. Nhận xét sơ bộ
print("""
🔹 Dataset gồm khoảng 10k khách hàng, 14 cột đặc trưng.
🔹 Biến mục tiêu: Exited (1 = Churn, 0 = Retained).
🔹 Dữ liệu khá sạch, không có missing values.
🔹 Các yếu tố ảnh hưởng lớn đến churn: Age, CreditScore, IsActiveMember, Geography.
""")


### 📌 Tổng kết ban đầu

- Dataset gồm khoảng **10.000 khách hàng**, **14 cột đặc trưng**.  
- Biến mục tiêu: **Exited**  
  - 1 → khách hàng **rời đi (Churn)**  
  - 0 → khách hàng **ở lại (Retained)**  
- Dữ liệu **khá sạch**, không có missing values.  
- Một số yếu tố có thể ảnh hưởng lớn đến khả năng churn:
  - `Age`, `CreditScore`, `IsActiveMember`, `Geography`.
